In [1]:
!pip install transformers
!pip install SentencePiece
!pip install pandas
!pip install tqdm

In [2]:
#DEFINE LANGUAGES
language1 = "en"
language2 = "ja"
file = "retrieval"

In [3]:
#COVID
import pandas as pd
import json

#retrieval
input_json = f"../input/cpgqa-dataset/cpgQA-{file}_train.json"
with open(input_json, "r", encoding = "utf-8") as json_file:
    objs = json.load(json_file)

questions = []
for i in range(len(objs)):
        question = objs[i]["question"]
        questions.append(question)
questions

['what are the components of biopsychosocial assessment?',
 'alternatives to opioids are recommended for what?',
 'what includes prior medical records?',
 'in a sample of non-treatment-seeking members of the military who were interviewed within three months of returning from afghanistan, how many reported using opioids?',
 'how much to take from days 11 to 20, when using morphine sr 15 mg tablets?',
 'what therapies are preferred for chronic pain?',
 'what should be done at every phase of treatment?',
 'what can cause a toxidrome?',
 'which conditions can complicate pain management?',
 'what to do if there are issues with the veteran obtaining outside prescriptions during the taper?',
 'which interventions are safe and have not been shown to increase morbidity or mortality?',
 'what to do if the patient is not willing to engage in sud therapy?',
 'what are the necessary risk mitigation strategies?',
 'why should an optimal approach to care should include a robust,  signature informed c

In [4]:
import torch
torch.cuda.empty_cache()
from transformers import pipeline

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-{language1}-jap")
tokenizer = AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{language1}-jap")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/998k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

In [6]:
model.to('cuda:0')
pipe = pipeline("translation", model=model, tokenizer=tokenizer, device = 0)

In [7]:
#COVID
from transformers import pipeline
from tqdm import tqdm

#FIRST TRANSLATION
first = []
for question in tqdm(questions):
    translated_str = pipe(question)[0]['translation_text']
    first.append(translated_str)

  1%|          | 10/877 [00:04<03:36,  4.01it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 877/877 [04:44<00:00,  3.08it/s]


In [8]:
torch.cuda.empty_cache()

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-ja-{language1}")
tokenizer = AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-ja-{language1}")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/764k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

In [10]:
model.to('cuda:0')
pipe = pipeline("translation", model=model, tokenizer=tokenizer, device = 0)

In [ ]:
#COVID
#SECOND TRANSLATION
second = []
for question in tqdm(first):
    translated_str = pipe(question)[0]['translation_text']
    second.append(translated_str)

  6%|▌         | 51/877 [00:11<03:31,  3.91it/s]

In [ ]:
#COVID -- replace question in json then save
for i in range(len(objs)):
    objs[i]["question"] = second[i]
with open("cpgQA-{}-{}-{}-{}.json".format(file,language1,language2,language1), 'w') as f:
                f.write(json.dumps(objs, indent = 4) + "\n")